In [ ]:
%reset -f

import random
from math import floor

fp_a = open('DIVIDE_DAT_A_S0I8F0.dat', 'w')
fp_b = open('DIVIDE_DAT_B_S0I8F0.dat', 'w')
fp_c = open('DIVIDE_DAT_C_S0I8F8.dat', 'w')
j_max = 0
k_max = 0
t_max = 0

DATA_WD = 8
for a in range(2 ** DATA_WD):
    for b in range(2 ** DATA_WD):
        if b != 0:
            # full precision
            DATA_SHIFT = DATA_WD
            DATA_SCALE = 2 ** DATA_SHIFT
            c_target = a / b
            c_target = floor(c_target * DATA_SCALE + 0.5)

            # directly use "/"
            c_calc = 0
            DATA_SHIFT = DATA_WD
            c_calc = (a << DATA_SHIFT) / b
            c_calc = floor(c_calc + 0.5)
            if (c_calc != c_target):
                print("ERROR @ directly use "/"")
                print(f"{a} / {b} = {c_target}")
                print(f"{a} / {b} = {c_calc}")
                break

            # directly use "/" (in a more hardware other way)
            c_calc = 0
            DATA_SHIFT = DATA_WD + 1
            c_calc = (a << DATA_SHIFT) // b
            c_calc = (c_calc + 1) >> 1
            if (c_calc != c_target):
                print("ERROR @ directly use "/"")
                print(f"{a} / {b} = {c_target}")
                print(f"{a} / {b} = {c_calc}")
                break

            # use remainder restoring method
            c_calc = 0
            DATA_SHIFT_T = DATA_WD + 1
            DATA_SHIFT_B = DATA_WD * 2
            t = a << DATA_SHIFT_T
            c_calc = 0
            for idxBit in range(DATA_SHIFT_B + 1):
                t = t - (b << DATA_SHIFT_B)
                if t < 0:
                    t = t + (b << DATA_SHIFT_B)
                    c_calc = c_calc << 1
                else:
                    c_calc = (c_calc << 1) + 1
                t = t << 1
            c_calc = (c_calc + 1) >> 1
            if (c_calc != c_target):
                print("ERROR @ remainder restoring method")
                print(f"{a} / {b} = {c_target}")
                print(f"{a} / {b} = {c_calc}")
                break

            # use remainder restoring method (in a more hardware other way)
            c_calc = 0
            NUMB_BIT   = DATA_WD * 2 + 1
            DATA_SHIFT = DATA_WD - 1
            DATA_MASK  = 2 ** DATA_SHIFT - 1
            t = a
            c_calc = 0
            for idxBit in range(NUMB_BIT):
                d = (t >> DATA_SHIFT) - b
                t = (d << DATA_SHIFT) | (t & DATA_MASK)
                if t < 0:
                    d = (t >> DATA_SHIFT) + b
                    t = (d << DATA_SHIFT) | (t & DATA_MASK)
                    c_calc = c_calc << 1
                else:
                    c_calc = (c_calc << 1) + 1
                t = t << 1
            c_calc = (c_calc + 1) >> 1
            if (c_calc != c_target):
                print("ERROR @ remainder restoring method (in a more hardware other way)")
                print(f"{a} / {b} = {c_target}")
                print(f"{a} / {b} = {c_calc}")
                break

            # use remainder non-restoring method
            c_calc = 0
            DATA_SHIFT_T = DATA_WD + 1
            DATA_SHIFT_B = DATA_WD * 2
            t = a << DATA_SHIFT_T
            t = t - (b << DATA_SHIFT_B)
            c_calc = 0
            for idxBit in range(DATA_SHIFT_B + 1):
                t = t * 2
                if t < 0:
                    t = t + (b << DATA_SHIFT_B)
                    c_calc = c_calc << 1
                else:
                    t = t - (b << DATA_SHIFT_B)
                    c_calc = (c_calc << 1) + 1
            c_calc = (c_calc + 1) >> 1
            if (c_calc != c_target):
                print("ERROR @ remainder non-restoring method")
                print(f"{a} / {b} = {c_target}")
                print(f"{a} / {b} = {c_calc}")
                break

            # use remainder non-restoring method (in a more hardware other way)
            c_calc = 0
            NUMB_BIT   = DATA_WD * 2 + 1
            DATA_SHIFT = DATA_WD - 1
            DATA_MASK  = 2 ** DATA_SHIFT - 1
            t = a
            d = (t >> DATA_SHIFT) - b
            t = (d << DATA_SHIFT) | (t & DATA_MASK)
            c_calc = 0
            for idxBit in range(NUMB_BIT):
                t = t << 1
                if t < 0:
                    d = (t >> DATA_SHIFT) + b
                    t = (d << DATA_SHIFT) | (t & DATA_MASK)
                    c_calc = c_calc << 1
                else:
                    d = (t >> DATA_SHIFT) - b
                    t = (d << DATA_SHIFT) | (t & DATA_MASK)
                    c_calc = (c_calc << 1) + 1
            c_calc = (c_calc + 1) >> 1
            if (c_calc != c_target):
                print("ERROR @ remainder non-restoring method (in a more hardware other way)")
                print(f"{a} / {b} = {c_target}")
                print(f"{a} / {b} = {c_calc}")
                break

            # use LUT
            c_calc = 0
            DATA_SHIFT = DATA_WD + 16
            DATA_SCALE = 2 ** DATA_SHIFT
            r = floor(1 / b * DATA_SCALE + 0.5) / DATA_SCALE
            c_calc = floor(a * r * (2 ** DATA_WD) + 0.5)
            if (c_calc != c_target):
                print("ERROR @ LUT")
                print(f"{a} / {b} = {c_target}")
                print(f"{a} / {b} = {c_calc}")
                break

            # use LUT (in a more hardware other way)
            c_calc = 0
            DATA_SHIFT = DATA_WD + 16
            DATA_SCALE = 2 ** DATA_SHIFT
            r = floor(1 / b * DATA_SCALE + 0.5)
            c_calc = a * r << DATA_WD >> (DATA_SHIFT - 1)
            c_calc = (c_calc + 1) >> 1
            if (c_calc != c_target):
                print("ERROR @ LUT (in a more hardware other way)")
                print(f"{a} / {b} = {c_target}")
                print(f"{a} / {b} = {c_calc}")
                break

            # binary search
            c_calc = 0
            NUMB_BIT = DATA_WD + 18
            DATA_SHIFT = DATA_WD + 16
            DATA_SCALE = 2 ** DATA_SHIFT
            r = 0
            for idxBit in range(NUMB_BIT):
                r = r + (2 ** (- idxBit))
                if r * b > 1:
                    r = r - (2 ** (- idxBit))
            r = floor(r * DATA_SCALE + 0.5) / DATA_SCALE
            c_calc = floor(a * r * (2 ** DATA_WD) + 0.5)
            if (c_calc != c_target):
                print("ERROR @ binary search")
                print(f"{a} / {b} = {c_target}")
                print(f"{a} / {b} = {c_calc}")
                break

            # use Newton-Raphson iteraion (with full precision)
            c_calc = 0
            NUMB_ITER = 12
            t = 1 / (2 ** DATA_WD)
            for idxItr in range(NUMB_ITER):
                t = t * (2 - b * t)
            c_calc = floor(a * t * (2 ** DATA_WD) + 0.5)
            if (c_calc != c_target):
                print("ERROR @ Newton-Raphson iteraion (with full precision)")
                print(f"{a} / {b} = {c_target}")
                print(f"{a} / {b} = {c_calc}")
                break

            # use Newton-Raphson iteraion
            c_calc = 0
            NUMB_ITER = 12
            DATA_SHIFT = DATA_WD + 16
            DATA_SCALE = 2 ** DATA_SHIFT
            t = 1 / (2 ** DATA_WD)
            for idxItr in range(NUMB_ITER):
                j = b * t
                j = floor(j * DATA_SCALE + 0.5) / DATA_SCALE
                if j_max < j:
                    j_max = j
                k = 2 - j
                if k_max < k:
                    k_max = k
                t = t * k
                t = floor(t * DATA_SCALE + 0.5) / DATA_SCALE
                if t_max < t:
                    t_max = t
            c_calc = floor(a * t * (2 ** DATA_WD) + 0.5)
            fp_a.write("{:x}\n".format(a))
            fp_b.write("{:x}\n".format(b))
            fp_c.write("{:x}\n".format(c_calc))
            if (c_calc != c_target):
                print("ERROR @ Newton-Raphson iteraion")
                print(f"{a} / {b} = {c_target}")
                print(f"{a} / {b} = {c_calc}")
                break
    if (c_calc != c_target):
        break
print(f"max of j, k, t is: {j_max}, {k_max}, {t_max}")
print("DONE!")
fp_a.close()
fp_b.close()
fp_c.close()